In [1]:
#Import necessary libraries
import pandas as pd
import numpy as np

In [2]:
#Import & transform the price data set
    #Import & sorting
prices = pd.read_csv('prices.csv')
prices = prices.sort_values(['product_id', 'updated_at']).reset_index(drop=True)

    #Derive effective day range of a specific price by self-joining
prices_merge = prices.merge(prices, left_on=['product_id', 'new_price'], right_on=['product_id', 'old_price'], how='outer', suffixes = ('_beg','_end'))
prices_merge['price'] = np.where(pd.isna(prices_merge['updated_at_beg']) == False, prices_merge['new_price_beg'], prices_merge['old_price_end'])
prices_merge['begin'] = np.where(pd.isna(prices_merge['updated_at_beg']) == False, prices_merge['updated_at_beg'], '9/1/18 00:00')
prices_merge['end'] = np.where(pd.isna(prices_merge['updated_at_end']) == False, prices_merge['updated_at_end'], '9/30/18 23:59')

    #Clean, format, and transform datetime the observations
prices_merge['begin'] = pd.to_datetime(prices_merge['begin'], format='%m/%d/%y %H:%M')
prices_merge['end'] = pd.to_datetime(prices_merge['end'], format='%m/%d/%y %H:%M')
prices_selected = prices_merge.get(['product_id', 'price', 'begin', 'end'])
condition = (prices_merge['begin'] < prices_merge['end'])
prices_final = prices_selected[condition].groupby(['product_id', 'price', 'begin'])['end'].min().to_frame().sort_values(['product_id','begin']).reset_index()

In [3]:
#Import & join sales data set
sales = pd.read_csv('sales.csv')
sales_joined = sales.merge(prices_final, on='product_id', how='left')
sales_joined['ordered_at'] = pd.to_datetime(sales_joined['ordered_at'], format='%m/%d/%y %H:%M')

In [4]:
#Filter sales_joined to identify the right price
filter_set = (sales_joined['ordered_at'] >= sales_joined['begin']) & (sales_joined['ordered_at'] < sales_joined['end'])
sales_filtered = sales_joined[filter_set].copy()

In [5]:
#Calculate revenue by product by price
sales_filtered['revenue'] = sales_filtered['quantity_ordered'] * sales_filtered['price']
revenue = sales_filtered.groupby(['product_id', 'price'])['revenue'].sum()

In [6]:
#Result
print(revenue)

product_id  price   
64          239000.0     956000.0
3954203     57500.0       57500.0
            60000.0      180000.0
            64000.0      640000.0
3998909     15500.0       15500.0
            16500.0      231000.0
            17000.0       34000.0
4085861     52000.0     1040000.0
            53500.0     2140000.0
            58000.0     2204000.0
            60000.0      180000.0
            62500.0     1812500.0
            67000.0      871000.0
Name: revenue, dtype: float64
